https://keras.io/preprocessing/image/

# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Hyperparameter Tunning

In [3]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (32, 32, 3)
num_classes = 10

## Preprocess

In [4]:
import pandas as pd

In [5]:
train_df = pd.read_csv('dataset/train_dataset.csv')
test_df = pd.read_csv('dataset/test_dataset.csv')

In [8]:
train_df.head()

,path,class_name
0,dataset/cifar/train\0_frog.png,frog
1,dataset/cifar/train\10000_automobile.png,automobile
2,dataset/cifar/train\10001_frog.png,frog
3,dataset/cifar/train\10002_frog.png,frog
4,dataset/cifar/train\10003_ship.png,ship


In [9]:
test_df.head()

,path,class_name
0,dataset/cifar/test\0_cat.png,cat
1,dataset/cifar/test\1000_dog.png,dog
2,dataset/cifar/test\1001_airplane.png,airplane
3,dataset/cifar/test\1002_ship.png,ship
4,dataset/cifar/test\1003_deer.png,deer


In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range = 0.3,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='path',
    y_col='class_name',
    target_size=input_shape[:2],
    batch_size=batch_size
)

Found 50000 validated image filenames belonging to 10 classes.


In [12]:
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='path',
    y_col='class_name',
    target_size=input_shape[:2],
    batch_size=batch_size
)

Found 10000 validated image filenames belonging to 10 classes.


## Build Model

In [13]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [15]:
hist = model.fit(train_generator, steps_per_epoch=len(train_generator),
         epochs=num_epochs,
         validation_data=test_generator,
         validation_steps=len(test_generator))

Epoch 1/10
1563/1563 [==============================] - 147s 94ms/step - loss: 1.8785 - accuracy: 0.2957 - val_loss: 1.5515 - val_accuracy: 0.4277
Epoch 2/10
1563/1563 [==============================] - 139s 89ms/step - loss: 1.6629 - accuracy: 0.3856 - val_loss: 1.5214 - val_accuracy: 0.4453
Epoch 3/10
1563/1563 [==============================] - 133s 85ms/step - loss: 1.5949 - accuracy: 0.4153 - val_loss: 1.5190 - val_accuracy: 0.4338
Epoch 4/10
1563/1563 [==============================] - 133s 85ms/step - loss: 1.5463 - accuracy: 0.4347 - val_loss: 1.5094 - val_accuracy: 0.4290
Epoch 5/10
1563/1563 [==============================] - 148s 94ms/step - loss: 1.5267 - accuracy: 0.4457 - val_loss: 1.7474 - val_accuracy: 0.3795
Epoch 6/10
1563/1563 [==============================] - 133s 85ms/step - loss: 1.5012 - accuracy: 0.4555 - val_loss: 1.5576 - val_accuracy: 0.4305
Epoch 7/10
1563/1563 [==============================] - 126s 81ms/step - loss: 1.4894 - accuracy: 0.4630 - val_loss: 1